In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_dir = r'C:\Python Projects\weather-recognition\images'
test_path = os.path.join(data_dir, 'test')
train_path = os.path.join(data_dir, 'train')
val_path = os.path.join(data_dir, 'val')

In [3]:
from matplotlib.image import imread

In [4]:
img = os.path.join(train_path, 'rain', '11.jpg')

In [5]:
dim1 = []
dim2 = []

for direct in os.listdir(test_path):
    for image_filename in os.listdir(os.path.join(test_path,direct)):
        img = imread(os.path.join(test_path, direct, image_filename))
        d1,d2,colors = img.shape
        dim1.append(d1)
        dim2.append(d2)

In [6]:
np.mean(d1)

360.0

In [7]:
np.mean(d2)

540.0

In [8]:
image_shape = (360, 540, 3)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
image_gen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, zoom_range=0.2, rescale=1/255)

In [11]:
image_gen.flow_from_directory(train_path)

Found 5484 images belonging to 11 classes.


In [12]:
image_gen.flow_from_directory(test_path)

Found 684 images belonging to 11 classes.


In [13]:
image_gen.flow_from_directory(val_path)

Found 682 images belonging to 11 classes.


In [14]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten

In [15]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(11, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 358, 538, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 179, 269, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 177, 267, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 88, 133, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 86, 131, 128)      73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 43, 65, 128)      0

In [17]:
from keras.callbacks import EarlyStopping

In [18]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [19]:
batch_size=16

In [20]:
train_image_gen = image_gen.flow_from_directory(train_path, target_size=image_shape[:2],batch_size=batch_size)

Found 5484 images belonging to 11 classes.


In [21]:
test_image_gen = image_gen.flow_from_directory(test_path, target_size=image_shape[:2],batch_size=batch_size, shuffle=False)

Found 684 images belonging to 11 classes.


In [22]:
val_image_gen = image_gen.flow_from_directory(val_path, target_size=image_shape[:2],batch_size=batch_size, shuffle=False)

Found 682 images belonging to 11 classes.


In [23]:
results = model.fit(train_image_gen, epochs=20, validation_data=val_image_gen, callbacks=[early_stop])

Epoch 1/20
343/343 [==============================] - 169s 483ms/step - loss: 1.8878 - accuracy: 0.3508 - val_loss: 1.5212 - val_accuracy: 0.5029
Epoch 2/20
343/343 [==============================] - 162s 473ms/step - loss: 1.4705 - accuracy: 0.5000 - val_loss: 1.3260 - val_accuracy: 0.5645
Epoch 3/20
343/343 [==============================] - 165s 481ms/step - loss: 1.3089 - accuracy: 0.5578 - val_loss: 1.3761 - val_accuracy: 0.4853
Epoch 4/20
343/343 [==============================] - 168s 490ms/step - loss: 1.2301 - accuracy: 0.5839 - val_loss: 1.1509 - val_accuracy: 0.6070
Epoch 5/20
343/343 [==============================] - 233s 680ms/step - loss: 1.1694 - accuracy: 0.6043 - val_loss: 1.1148 - val_accuracy: 0.6056
Epoch 6/20
343/343 [==============================] - 172s 503ms/step - loss: 1.1292 - accuracy: 0.6112 - val_loss: 1.1002 - val_accuracy: 0.6129
Epoch 7/20
343/343 [==============================] - 168s 490ms/step - loss: 1.1036 - accuracy: 0.6220 - val_loss: 0.9814 -

In [27]:
pred = model.predict(test_image_gen)

43/43 [==============================] - 20s 468ms/step


In [35]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [36]:
model.evaluate(test_image_gen, batch_size=16)

43/43 [==============================] - 19s 443ms/step - loss: 0.8269 - accuracy: 0.7339


[0.8268906474113464, 0.7339181303977966]